In [ ]:
#Import necessary libraries and modules from TensorFlow and Keras

import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM
from tensorflow.keras.preprocessing import sequence

In [ ]:
#Download and extract the IMDB dataset.

!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  23.7M      0  0:00:03  0:00:03 --:--:-- 23.7M


In [ ]:
#Define the maximum number of features and the maximum length of sequences

max_features = 10000
maxlen = 150

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#Custom layer representing one layer of a transformer encoder
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)

#Dense projection network
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

#Define how inputs are processed by the layer
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)

        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

 #Serialize the layer's configuration
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [ ]:
#Define parameters for the Transformer encoder layer

vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32


#Defined a model architecture using an embedding layer followed by a Transformer encoder layer and other dense layers.
#Compiling the model with optimizer, loss function, and metrics

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Trans  (None, None, 256)         543776    
 formerEncoder)                                                  
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

In [ ]:
import string

#Defining a class Vectorizer with a method "standardize" to preprocess text by converting it to lowercase and removing punctuation.
class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)

In [ ]:
#Define a method "tokenize"

def tokenize(self, text):
        text = self.standardize(text)
        return text.split()

In [ ]:
#creating a vocabulary from the dataset through a method "make_vocabulary"

def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

In [ ]:
#Defining a method "encode" to encode text using the created vocabulary

def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

In [ ]:
#Decoding integer sequences back to text

def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

In [ ]:
import re
import string
import tensorflow as tf

In [ ]:
#Using a custom standardization function for text preprocessing using TensorFlow

def custom_standardization_fn(string_tensor):
    lowercase_string = tf.strings.lower(string_tensor)
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation)}]", "")

In [ ]:
#Using a custom split function for text preprocessing

def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)


In [ ]:
#Removing the unsupervised training data directory, displaying the content of a positive review file

!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt
import os, pathlib, shutil, random

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  22.9M      0  0:00:03  0:00:03 --:--:-- 22.9M
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [ ]:
#Defining batch size and directories for the dataset

batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

In [ ]:
#Moving a portion of training data to the validation directory for each category
#Here, I am making sure the directory and file exist

for category in ("neg", "pos"):

    if not os.path.exists(val_dir / category):
        os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:

        if not os.path.exists(val_dir / category / fname):
            shutil.move(train_dir / category / fname,
                        val_dir / category / fname)

In [ ]:
from tensorflow import keras
batch_size = 32

In [ ]:
#Creating TensorFlow datasets for training, validation, and testing

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 25000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
#Displaying the shapes and dtypes of the first batch

for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'It was once suggested by Pauline Kael, never a fan, that Cassavetes thought not like a director, but like an actor. What Kael meant was his supposed lack of sophistication as a filmmaker; to take that comparison further, to me, it never feels like Cassavetes is directing himself in a film, it feels like Cassavetes implanting himself inside his own creation, like Orson Welles. Cassavetes is just as much of a genius as Welles, but far more important as a true artist (as opposed to a technician or rhetorician). This is like a cross between Italian passion (though Cassavetes was actually Greek) and Scandinavian introversion. Never before have inner demons been so exposed physically.<br /><br />It\'s about the mystery of becoming, performing, and acting. Like a haunted Skip James record, it\'s got the echoes of ghosts all around. Rowlands\' breakdowns, which are st

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
#Creating a dataset containing only text inputs for training

text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot",
)
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

In [ ]:
#Using the text_vectorization function to convert text data into binary 1-gram representations

#Here, the use of "num_parallel_calls=4" argument indicates mapping done in parallel using four threads

binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [ ]:
#Data Exploration in binary_1gram_train_ds

for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
#Function to Build and Compile a Feedforward Neural Network Model
#max_tokens (int) indicates maximum number of tokens in each input, while sequence.hidden_dim (int) indicates dimensionality of the hidden layer.

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
#Creating and summarizing a simple dense model.

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True)
]

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_6 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#Training the model using the integer encoded training and validation dataset

model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

#Loading the model and evaluating its performance on the test dataset
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Epoch 1/10
782/782 [==============================] - 25s 31ms/step - loss: 0.3940 - accuracy: 0.8357 - val_loss: 0.2205 - val_accuracy: 0.9194
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2755 - accuracy: 0.8986 - val_loss: 0.1841 - val_accuracy: 0.9356
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2517 - accuracy: 0.9127 - val_loss: 0.1710 - val_accuracy: 0.9456
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2378 - accuracy: 0.9202 - val_loss: 0.1557 - val_accuracy: 0.9494
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2336 - accuracy: 0.9229 - val_loss: 0.1484 - val_accuracy: 0.9538
Epoch 6/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2297 - accuracy: 0.9250 - val_loss: 0.1537 - val_accuracy: 0.9544
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2273 - accuracy: 0.9271 - val_loss: 0.1432 - val_accuracy: 0.9544
Epoc

In [ ]:
#Configuring TextVectorization Parameters

text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

In [ ]:
#Training and testing the binary bigram model

text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [ ]:
#Initializing Model and Setting Up Checkpoints

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                    save_best_only=True)
]

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_8 (Dense)             (None, 16)                320016    
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#Training the Model and Evaluating Test Accuracy

model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")


Epoch 1/10
782/782 [==============================] - 14s 18ms/step - loss: 0.3688 - accuracy: 0.8512 - val_loss: 0.1890 - val_accuracy: 0.9362
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2433 - accuracy: 0.9156 - val_loss: 0.1540 - val_accuracy: 0.9444
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2183 - accuracy: 0.9294 - val_loss: 0.1271 - val_accuracy: 0.9596
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2059 - accuracy: 0.9364 - val_loss: 0.1115 - val_accuracy: 0.9640
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.1991 - accuracy: 0.9412 - val_loss: 0.1094 - val_accuracy: 0.9622
Epoch 6/10
782/782 [==============================] - 6s 7ms/step - loss: 0.1973 - accuracy: 0.9446 - val_loss: 0.1070 - val_accuracy: 0.9646
Epoch 7/10
782/782 [==============================] - 6s 7ms/step - loss: 0.1901 - accuracy: 0.9462 - val_loss: 0.0985 - val_accuracy: 0.9688
Epoc

In [ ]:
#Configuring the TextVectorization layer to return token counts

text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count"
)

In [ ]:
#Configuring TextVectorization to return TF-IDF-weighted outputs

text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)

In [ ]:
#Training and testing the TF-IDF bigram model

text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)



In [ ]:
#Initializing the Model and Summary
model = get_model()
model.summary()

#Model Checkpoint Configuration
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only=True)
]

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_10 (Dense)            (None, 16)                320016    
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#Model Training and Evaluation on Test Data

model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")


Epoch 1/10
782/782 [==============================] - 13s 16ms/step - loss: 0.5070 - accuracy: 0.7670 - val_loss: 0.3083 - val_accuracy: 0.8842
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3605 - accuracy: 0.8458 - val_loss: 0.1952 - val_accuracy: 0.9232
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3236 - accuracy: 0.8568 - val_loss: 0.1831 - val_accuracy: 0.9390
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2999 - accuracy: 0.8645 - val_loss: 0.1725 - val_accuracy: 0.9372
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2852 - accuracy: 0.8691 - val_loss: 0.1740 - val_accuracy: 0.9276
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2730 - accuracy: 0.8788 - val_loss: 0.1467 - val_accuracy: 0.9476
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2637 - accuracy: 0.8864 - val_loss: 0.1443 - val_accuracy: 0.9468
Epo

In [ ]:
#Model Inference and Prediction

inputs = keras.Input(shape=(1,), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)
import tensorflow as tf
raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")

99.74 percent positive


In [ ]:
#Processing words as a sequence: The sequence model approach

#Downloading the data

!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  15.7M      0  0:00:05  0:00:05 --:--:-- 16.1M


In [ ]:
#Preparing the data

import os, pathlib, shutil, random
from tensorflow import keras


In [ ]:
#Setting up directories

batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

In [ ]:
for category in ("neg", "pos"):
    # Check if the directory already exists
    if not os.path.exists(val_dir / category):
        os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        # Check if the file already exists in the destination directory
        if not os.path.exists(val_dir / category / fname):
            shutil.move(train_dir / category / fname,
                        val_dir / category / fname)

In [ ]:
#Creating Datasets from Directories

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)


Found 25000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
#Preparing integer sequence datasets

from tensorflow.keras import layers


In [ ]:
#Text Vectorization: Initialization and Adaptation

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

In [ ]:
#Integer Encoding Mapping for Datasets

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [ ]:
#A sequence model built on one-hot encoded vector sequences

import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirection  (None, 64)                5128448   
 al)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5128513 (19.56 MB)
Trainable params: 5128513 (19.56 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [ ]:
from tensorflow import keras

In [ ]:
from tensorflow.keras import layers

In [ ]:
#Creating a simple neural network model for binary classification tasks to specify the maximum number of input tokens and the size of the hidden layer

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
# Define the model
model = get_model()


In [ ]:
#Callback for Model Checkpointing

callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]

In [ ]:
import os
import random
import shutil
import pathlib
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf



In [ ]:
# Define the base directory
base_dir = pathlib.Path("aclImdb")

In [ ]:
# Define the directories for train, validation, and test sets
train_dir = base_dir / "train"
val_dir = base_dir / "val"
test_dir = base_dir / "test"

In [ ]:
# Create directories for validation set
for category in ("neg", "pos"):
    os.makedirs(val_dir / category, exist_ok=True)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname, val_dir / category / fname)


In [ ]:
#Data Preparation for Text Classification

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    train_dir, batch_size=batch_size, validation_split=0.2, subset="training", seed=1337
)
val_ds = keras.utils.text_dataset_from_directory(
    train_dir, batch_size=batch_size, validation_split=0.2, subset="validation", seed=1337
)
test_ds = keras.utils.text_dataset_from_directory(
    test_dir, batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Using 16000 files for training.
Found 20000 files belonging to 2 classes.
Using 4000 files for validation.
Found 25000 files belonging to 2 classes.


In [ ]:
#Defining the TextVectorization layer
max_features = 20000
sequence_length = 600

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length,
)


In [ ]:
#Adapting the TextVectorization layer on the training dataset
train_text = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)


In [ ]:
#Define the model
embedding_dim = 128
model = keras.Sequential([
    vectorize_layer,
    layers.Embedding(max_features + 1, embedding_dim),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

In [ ]:
#Compile the model
model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

In [ ]:
#Train the model
model.fit(train_ds, validation_data=val_ds, epochs=10)

#Evaluate the model
print("Evaluate on test data")
results = model.evaluate(test_ds)
print("Test loss, Test accuracy:", results)

Epoch 1/10
500/500 [==============================] - 413s 816ms/step - loss: 0.6190 - accuracy: 0.5858 - val_loss: 0.4190 - val_accuracy: 0.8320
Epoch 2/10
500/500 [==============================] - 383s 767ms/step - loss: 0.2934 - accuracy: 0.8811 - val_loss: 0.3118 - val_accuracy: 0.8783
Epoch 3/10
500/500 [==============================] - 406s 813ms/step - loss: 0.1912 - accuracy: 0.9300 - val_loss: 0.3127 - val_accuracy: 0.8708
Epoch 4/10
500/500 [==============================] - 386s 772ms/step - loss: 0.1307 - accuracy: 0.9554 - val_loss: 0.3325 - val_accuracy: 0.8788
Epoch 5/10
500/500 [==============================] - 382s 764ms/step - loss: 0.0873 - accuracy: 0.9721 - val_loss: 0.3753 - val_accuracy: 0.8832
Epoch 6/10
500/500 [==============================] - 406s 813ms/step - loss: 0.0581 - accuracy: 0.9836 - val_loss: 0.4378 - val_accuracy: 0.8742
Epoch 7/10
500/500 [==============================] - 385s 770ms/step - loss: 0.0379 - accuracy: 0.9891 - val_loss: 0.4297 -

In [ ]:
max_tokens = 20000  # Defining the maximum number of tokens in your vocabulary
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
#Model definition with embedding layer for text classification

embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])


In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [ ]:
#Directory paths for training, validation, and test data

train_dir = base_dir / "train"
val_dir = base_dir / "val"
test_dir = base_dir / "test"



In [ ]:
#Model architecture definition with embedding layer, bidirectional LSTM, dropout, and dense layer

embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

In [ ]:
#Model input and layers configuration: embedding, bidirectional LSTM, dropout, and dense layers

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

In [ ]:
#Model compilation with RMSprop optimizer, binary crossentropy loss, and accuracy metric

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
